# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
def build_classifiers():
    
    # fill this part
    classifiers = [LinearRegression(), KNeighborsClassifier(), SVC(), DecisionTreeClassifier(), GaussianNB(), QuadraticDiscriminantAnalysis()]
    for classifier in classifiers:
        classifier.fit(data_set, labels)

    return classifiers # and here

In [4]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [5]:
from itertools import permutations

classifiers = build_classifiers()
classifiers_results = {}

for classifiers_permutation in permutations(classifiers, r=3):
    predicted = build_stacked_classifier(classifiers_permutation)
    accuracy = accuracy_score(test_labels, predicted)
    permutation_name = ' '.join(str(classifier) for classifier in classifiers_permutation)
    classifiers_results[permutation_name] = accuracy

sorted_results = dict(sorted(classifiers_results.items(), key=lambda x: x[1], reverse=True))

for permutation_name, accuracy in sorted_results.items():
    print(f'Accuracy: {accuracy}: {permutation_name}')

Accuracy: 0.9: DecisionTreeClassifier() KNeighborsClassifier() QuadraticDiscriminantAnalysis()
Accuracy: 0.9: GaussianNB() KNeighborsClassifier() QuadraticDiscriminantAnalysis()
Accuracy: 0.9: QuadraticDiscriminantAnalysis() KNeighborsClassifier() DecisionTreeClassifier()
Accuracy: 0.85: KNeighborsClassifier() SVC() GaussianNB()
Accuracy: 0.85: KNeighborsClassifier() SVC() QuadraticDiscriminantAnalysis()
Accuracy: 0.85: KNeighborsClassifier() DecisionTreeClassifier() QuadraticDiscriminantAnalysis()
Accuracy: 0.85: KNeighborsClassifier() GaussianNB() QuadraticDiscriminantAnalysis()
Accuracy: 0.85: KNeighborsClassifier() QuadraticDiscriminantAnalysis() DecisionTreeClassifier()
Accuracy: 0.85: SVC() KNeighborsClassifier() GaussianNB()
Accuracy: 0.85: SVC() KNeighborsClassifier() QuadraticDiscriminantAnalysis()
Accuracy: 0.85: GaussianNB() KNeighborsClassifier() DecisionTreeClassifier()
Accuracy: 0.85: GaussianNB() SVC() KNeighborsClassifier()
Accuracy: 0.8: KNeighborsClassifier() SVC() De

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

def calculate_accuracy_vector(predicted, labels):
    return [1 if predicted[i]==labels[i] else 0 for i in range(len(predicted))]

Fill the two functions below:

In [7]:
def set_new_weights(model):
    result = np.array(calculate_accuracy_vector(model.predict(test_set), test_labels))
    return (result + 1) / result.sum()

Train the classifier with the code below:

In [8]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

# print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

Set the validation data set:

In [9]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [10]:
def get_prediction(x):
    predictions = np.array([classifier.predict(x) for classifier in classifiers])
    return [np.argmax(np.bincount(predictions.flatten()))]

Test it:

In [11]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
